In [1]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
import numpy as np
from quantfreedom.utils.caching import clear_cache
from quantfreedom.base.base import  plot_trades_all_info
from quantfreedom.nb.simulate import simulate_up_to_6
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.evaluators.evaluators import eval_is_below

np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv(
    '../data/30min.csv', index_col='time')

rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=15,
)

rsi_eval = eval_is_below(
    rsi_ind,
    np.arange(40, 61, 10),
)


In [2]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=rsi_eval.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    # sl_pcts=[2,3,4],
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2,5,1),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1,4,1),
    tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [3]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.iloc[3009:3013]

,order_id,settings_id,bar,size_value,price,avg_entry,fees_paid,order_type,real_pnl,equity,sl_prices,tsl_prices,tp_prices
3009,3009,2,4280,"1,996.75","26,413.58","26,336.85",2.40,LongTSL,3.42,"2,058.87",NaN,"26,413.58","32,879.06"
3010,3010,2,4281,500.02,"26,563.50","26,563.50",NaN,LongEntry,NaN,"2,058.87",NaN,"25,500.96","33,162.01"
3011,3011,2,4282,500.02,"26,504.50","26,533.97",NaN,LongEntry,NaN,"2,058.87",NaN,"25,472.61","33,125.14"
3012,3012,2,4283,500.02,"26,250.00","26,438.63",NaN,LongEntry,NaN,"2,058.87",NaN,"25,381.09","33,006.12"


In [4]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

,0,1,2
total_trades,195.00,155.00,107.00
gains_pct,-22.95,57.47,171.01
win_rate,37.95,46.45,49.53
to_the_upside,0.67,0.11,0.27
total_pnl,-229.47,574.66,"1,710.10"
ending_eq,770.53,"1,574.66","2,710.10"
settings_id,0,1,2
max_equity_risk_pct,4.00,4.00,4.00
risk_rewards,3.00,5.00,6.00
size_pct,1.00,1.00,1.00


In [5]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [6]:
clear_cache()